In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision import datasets
import LoggerYN as YN
import scipy.io as sio
import utilsYN as uYN
import numpy as np
import time
import datetime


In [ ]:

def initParameters(dataset,batchSize,numClasses,epochs,learningRate,momentum,weightDecay):
    global Dataset    
    global pbatchSize
    global pnumClasses
    global pEpochs
    global pLearningRate
    global pMomentum
    global pWeightDecay
    Dataset = dataset
    pbatchSize = batchSize
    pnumClasses = numClasses
    pEpochs = epochs
    pLearningRate = learningRate
    pMomentum = momentum
    pWeightDecay = weightDecay

    device = torch.device("cpu")
    kwargs =  {}
    
    return device,kwargs

def weights_init(m):
    if isinstance(m, nn.Conv2d):    
        nn.init.xavier_uniform(m.weight.data)

def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]
        
        
def NormalizeData(x_train,x_test):
        x_train /= 255
        x_test /= 255
        return x_train, x_test
    


def MNIST_Loader(kwargs):
    MNIST_train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),batch_size=pbatchSize, shuffle=True, **kwargs)
    
    MNIST_test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])), shuffle=True, **kwargs)
    
    
    return MNIST_train_loader, MNIST_test_loader 
                  

def MNIST_train(model, device, train_loader, optimizer, epoch):
    model.train()
    train_loss = 0
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        input = model(data)
        loss = F.cross_entropy(input, target)
        train_loss += loss.sum().item()
        pred = input.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        
        loss.backward()
        optimizer.step()
        global pbatchSize
        
        if batch_idx % pbatchSize == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\t Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    
    train_loss /= len(train_loader)
    print('\nTrain_Loss: {:.6f} , Train_accuracy: {:.6f}'.format(train_loss,correct / len(train_loader.dataset)), end="")
            
def MNIST_test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            input = model(data)
            test_loss += F.cross_entropy(input, target, reduction='sum').item() # sum up batch loss
            pred = input.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader)
    print(', Test_loss: {:.4f}, Test_accuracy: {:.5f}\n'.format(test_loss,correct / len(test_loader.dataset)))
    

class MNIST_ConvNet(nn.Module):
    def __init__(self):
        super(MNIST_ConvNet, self).__init__()
        self.conv1 = nn.Conv2d( 1, 32,  kernel_size=(3))
        self.conv2 = nn.Conv2d( 32, 64, kernel_size=(3, 3))
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))  
        x = F.max_pool2d(x , kernel_size= (2, 2))
        x = F.dropout(x, 0.25) 
        x = x.view(x.size(0), -1)
        x= F.relu(self.fc1(x))        
        x = F.dropout(x,0.5)
        x = self.fc2(x)
        y = F.log_softmax(x)
        return y


    
def RunMNIST(dataset,batchSize,numClasses,epochs,learningRate,momentum,weightDecay):
    device, kwargs = initParameters(dataset,batchSize,numClasses,epochs,learningRate,momentum,weightDecay)
    MNIST_train_loader, MNIST_test_loader = MNIST_Loader(kwargs)
    MNIST_model = MNIST_ConvNet().to(device)
    MNIST_model.apply(weights_init)
    optimizer = optim.SGD(MNIST_model.parameters(), lr=pLearningRate, momentum=pMomentum, weight_decay=pWeightDecay)
    
    print("MNIST Training Started.....")
    memT,cpuT,gpuT = YN.StartLogger("Pytorch","MNIST") 
    start = time.time()
    for epoch in range(1,pEpochs + 1):  
        MNIST_train( MNIST_model, device, MNIST_train_loader, optimizer, epoch)
        MNIST_test(MNIST_model, device, MNIST_test_loader)
    end = time.time()
    print(str(datetime.timedelta(seconds=end-start)))
    YN.EndLogger(memT,cpuT,gpuT)
    print("MNIST Training Finished.....")
    
def CIFAR10_Loader(kwargs):

    
    CIFAR10_train_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('data', train=True, download=True,
                       transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                        ])), batch_size=pbatchSize, shuffle=True, **kwargs)
    CIFAR10_test_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                       ])), shuffle=True, **kwargs)
    CIFAR10_classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

    return CIFAR10_train_loader, CIFAR10_test_loader,CIFAR10_classes


    
def CIFAR10_train(epoch,criterion,optimizer,CIFAR10_trainloader,device,CIFAR10_model):
    CIFAR10_model.train()
    train_loss = 0
    correct = 0
    for batch_idx, (data, targets) in enumerate(CIFAR10_trainloader):
        data, targets = data.to(device), targets.to(device)
        optimizer.zero_grad()
        input = CIFAR10_model(data)
        loss = F.cross_entropy(input, targets)
        
        train_loss += loss.sum().item()
        pred = input.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(targets.view_as(pred)).sum().item()
        
        loss.backward()
        optimizer.step()
        global pbatchSize
        if batch_idx % pbatchSize == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(CIFAR10_trainloader.dataset),
                100. * batch_idx / len(CIFAR10_trainloader), loss.item()))

    train_loss /= len(CIFAR10_trainloader)
    print('\nTrain_Loss: {:.6f} , Train_accuracy: {:.6f}'.format(train_loss,correct / len(CIFAR10_trainloader.dataset)), end="")    


def CIFAR10_test(epoch, CIFAR10_testloader,CIFAR10_model, device,criterion):
    CIFAR10_model.eval()
    test_loss = 0
    correct = 0
    total=0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(CIFAR10_testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            input = CIFAR10_model(inputs)
            test_loss_temp = criterion(input, targets)
            test_loss += test_loss_temp.item()
            _, predicted = input.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()  
    
    test_loss /= len(CIFAR10_testloader)
    print(', Test_loss: {:.4f}, Test_accuracy: {:.5f}\n'.format(test_loss,correct / len(CIFAR10_testloader.dataset)))
    
      


class CIFAR10_ConvNet(nn.Module):
    def __init__(self):
        super(CIFAR10_ConvNet, self).__init__()
        self.conv1 = nn.Conv2d( 3, 32,  kernel_size=(3),padding=1,stride=1)
        self.conv2 = nn.Conv2d( 32, 32, kernel_size=(3, 3))
        self.conv3 = nn.Conv2d( 32, 64,  kernel_size=(3),padding=1,stride=1)
        self.conv4 = nn.Conv2d( 64, 64, kernel_size=(3, 3))
        
        self.fc1 = nn.Linear(2304, 512)
        self.fc2 = nn.Linear(512, 10)
            

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))  
        x = F.max_pool2d(x , kernel_size= (2, 2))
        x = F.dropout(x, 0.25)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))  
        x = F.max_pool2d(x , kernel_size= (2, 2))
        x = F.dropout(x, 0.25)
                
        #Flatten  
        x = x.view(x.size(0), -1)
        x= F.relu(self.fc1(x))    
        x = F.dropout(x,0.5)
        x = self.fc2(x)
        y = F.log_softmax(x)
        return y
    
    
def RunCIFAR10(dataset,batchSize,numClasses,epochs,learningRate,momentum,weightDecay):
    device,kwargs = initParameters(dataset,batchSize,numClasses,epochs,learningRate,momentum,weightDecay)
    CIFAR10_trainloader, CIFAR10_testloader, CIFAR10_classes = CIFAR10_Loader(kwargs)
    CIFAR10_model = CIFAR10_ConvNet()
    CIFAR10_model = CIFAR10_model.to(device)
    CIFAR10_model.apply(weights_init)
  
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(CIFAR10_model.parameters(), lr=pLearningRate, momentum=pMomentum, weight_decay=pWeightDecay)

    print("CIFAR10 Training Started.....")
    start = time.time()
    memT,cpuT,gpuT = YN.StartLogger("Pytorch","CIFAR10")     
    for epoch in range(1, pEpochs+1):
        CIFAR10_train(epoch,criterion,optimizer,CIFAR10_trainloader,device,CIFAR10_model)
        CIFAR10_test(epoch,CIFAR10_testloader,CIFAR10_model,device,criterion)
    end = time.time()
    print(str(datetime.timedelta(seconds=end-start)))
    YN.EndLogger(memT,cpuT,gpuT)
    print("CIFAR10 Training Finished.....")

def CIFAR100_Loader(kwargs):
    

    print('==> Preparing dataset CIFAR 100')
    
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    dataloader = datasets.CIFAR100
    
    trainset = dataloader(root='./data', train=True, download=True, transform=transform_train)
    CIFAR100_train_loader = torch.utils.data.DataLoader(trainset, batch_size=pbatchSize, shuffle=True,**kwargs)

    testset = dataloader(root='./data', train=False, download=False, transform=transform_test)
    CIFAR100_test_loader = torch.utils.data.DataLoader(testset, batch_size=pbatchSize ,shuffle=False, **kwargs)
    
    
    CIFAR100_classes = (
    'beaver', 'dolphin', 'otter', 'seal', 'whale',
    'aquarium fish', 'flatfish', 'ray', 'shark', 'trout',
    'orchids', 'poppies', 'roses', 'sunflowers', 'tulips',
    'bottles', 'bowls', 'cans', 'cups', 'plates',
    'apples', 'mushrooms', 'oranges', 'pears', 'sweet peppers',
    'clock', 'computer keyboard', 'lamp', 'telephone', 'television',
    'bed', 'chair', 'couch', 'table', 'wardrobe',
    'bee', 'beetle', 'butterfly', 'caterpillar', 'cockroach',
    'bear', 'leopard', 'lion', 'tiger', 'wolf',
    'bridge', 'castle', 'house', 'road', 'skyscraper',
    'cloud', 'forest', 'mountain', 'plain', 'sea',
    'camel', 'cattle', 'chimpanzee', 'elephant', 'kangaroo',
    'fox', 'porcupine', 'possum', 'raccoon', 'skunk',
    'crab', 'lobster', 'snail', 'spider', 'worm',
    'baby', 'boy', 'girl', 'man', 'woman',
    'crocodile', 'dinosaur', 'lizard', 'snake', 'turtle',
    'hamster', 'mouse', 'rabbit', 'shrew', 'squirrel',
    'maple', 'oak', 'palm', 'pine', 'willow',
    'bicycle', 'bus', 'motorcycle', 'pickup truck', 'train',
    'lawn-mower', 'rocket', 'streetcar', 'tank', 'tractor')

    return CIFAR100_train_loader, CIFAR100_test_loader,CIFAR100_classes



    
def CIFAR100_train(epoch,criterion,optimizer,CIFAR100_trainloader,device,CIFAR100_model):
    CIFAR100_model.train()
    train_loss = 0
    correct = 0
    for batch_idx, (data, targets) in enumerate(CIFAR100_trainloader):
        data, targets = data.to(device), targets.to(device)
        optimizer.zero_grad()
        input = CIFAR100_model(data)
        loss = F.cross_entropy(input, targets)
        
        train_loss += loss.sum().item()
        pred = input.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(targets.view_as(pred)).sum().item()
        
        loss.backward()
        optimizer.step()
        global pbatchSize
        if batch_idx % pbatchSize == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(CIFAR100_trainloader.dataset),
                100. * batch_idx / len(CIFAR100_trainloader), loss.item()))
    train_loss /= len(CIFAR100_trainloader)
    print('\nTrain_Loss: {:.6f} , Train_accuracy: {:.6f}'.format(train_loss,correct / len(CIFAR100_trainloader.dataset)), end="")            

def CIFAR100_test(epoch, CIFAR100_testloader,CIFAR100_model, device,criterion):
    CIFAR100_model.eval()
    test_loss = 0
    correct = 0
    total=0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(CIFAR100_testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            input = CIFAR100_model(inputs)
            test_loss_temp = criterion(input, targets)
            test_loss += test_loss_temp.item()
            _, predicted = input.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()  

    test_loss /= len(CIFAR100_testloader)
    print(', Test_loss: {:.4f}, Test_accuracy: {:.5f}\n'.format(test_loss,correct / len(CIFAR100_testloader.dataset)))    


    
class CIFAR100_ConvNet(nn.Module):
    def __init__(self):
        super(CIFAR100_ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 128,  kernel_size=(3,3),padding=1,stride=1)
        self.conv2 = nn.Conv2d( 128, 128, kernel_size=(3, 3))
        self.conv3 = nn.Conv2d( 128, 256, kernel_size=(3, 3),padding=1,stride=1)
        self.conv4 = nn.Conv2d( 256, 256, kernel_size=(3, 3))
        self.conv5 = nn.Conv2d( 256, 512, kernel_size=(3, 3),padding=1,stride=1)
        self.conv6 = nn.Conv2d( 512, 512, kernel_size=(3, 3))
         
        self.fc1 = nn.Linear(2048, 1024)
        self.fc2 = nn.Linear(1024, 100)

            

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x , kernel_size= (2, 2), stride=2)
        x = F.dropout(x,0.1)
        
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(x , kernel_size= (2, 2), stride=2)
        x = F.dropout(x,0.25)
        
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = F.max_pool2d(x , kernel_size= (2, 2), stride=2)
        x = F.dropout(x,0.5)     
        
        #Flatten  
        x = x.view(x.size(0), -1)
        x= F.relu(self.fc1(x))
        x = F.dropout(x,0.5)   
        x = self.fc2(x)
        y = torch.nn.Softmax(x)
        return x
    
def RunCIFAR100(dataset,batchSize,numClasses,epochs,learningRate,momentum,weightDecay):
    device,kwargs = initParameters(dataset,batchSize,numClasses,epochs,learningRate,momentum,weightDecay)
    CIFAR100_trainloader, CIFAR100_testloader, CIFAR100_classes = CIFAR100_Loader(kwargs)
    CIFAR100_model = CIFAR100_ConvNet()
    CIFAR100_model = CIFAR100_model.to(device)
    CIFAR100_model.apply(weights_init)
  
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(CIFAR100_model.parameters(), lr=pLearningRate, momentum=pMomentum, weight_decay=pWeightDecay)
    
    print("CIFAR100 Training Started.....")
    memT,cpuT,gpuT = YN.StartLogger("Pytorch","CIFAR100")
    start = time.time()
    for epoch in range(1, pEpochs+1):
        CIFAR100_train(epoch,criterion,optimizer,CIFAR100_trainloader,device,CIFAR100_model)
        CIFAR100_test(epoch,CIFAR100_testloader,CIFAR100_model,device,criterion)
    end = time.time()
    print(str(datetime.timedelta(seconds=end-start)))
    YN.EndLogger(memT,cpuT,gpuT)
    print("CIFAR100 Training Finished.....")

    

    
def SVHN_Loader(kwargs):
    SVHN_train_loader = torch.utils.data.DataLoader(
        datasets.SVHN('data', split='extra', download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=pbatchSize, shuffle=True, **kwargs)
    SVHN_test_loader = torch.utils.data.DataLoader(
        datasets.SVHN('data',split='test',download=True, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])), shuffle=True, **kwargs)
    
    
    return SVHN_train_loader, SVHN_test_loader 


def MNIST_train(model, device, train_loader, optimizer, epoch):
    model.train()
    train_loss = 0
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        input = model(data)
        loss = F.cross_entropy(input, target)
        train_loss += loss.sum().item()
        pred = input.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        
        loss.backward()
        optimizer.step()
        global pbatchSize
        
        if batch_idx % pbatchSize == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\t Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    
    train_loss /= len(train_loader)
    print('\nTrain_Loss: {:.6f} , Train_accuracy: {:.6f}'.format(train_loss,correct / len(train_loader.dataset)), end="")
            
def MNIST_test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            input = model(data)
            test_loss += F.cross_entropy(input, target, reduction='sum').item() # sum up batch loss
            pred = input.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader)
    print(', Test_loss: {:.4f}, Test_accuracy: {:.5f}\n'.format(test_loss,correct / len(test_loader.dataset)))    

def SVHN_train(model, device, train_loader, optimizer, epoch):
    model.train()
    train_loss = 0
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        input = model(data)
        loss = F.cross_entropy(input, target)
        
        train_loss += loss.sum().item()
        pred = input.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        
        loss.backward()
        optimizer.step()
        
        if batch_idx % pbatchSize == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\t Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    
    train_loss /= len(train_loader)
    print('\nTrain_Loss: {:.6f} , Train_accuracy: {:.6f}'.format(train_loss,correct / len(train_loader.dataset)), end="")        
        

def SVHN_test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            input = model(data)
            test_loss += F.cross_entropy(input, target, reduction='sum').item() # sum up batch loss
            pred = input.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader)
    print(', Test_loss: {:.4f}, Test_accuracy: {:.5f}\n'.format(test_loss,correct / len(test_loader.dataset)))        
    
def RunSVHN(dataset,batchSize,numClasses,epochs,learningRate,momentum,weightDecay):
    device, kwargs = initParameters(dataset,batchSize,numClasses,epochs,learningRate,momentum,weightDecay)
    SVHN_train_loader, SVHN_test_loader = SVHN_Loader(kwargs)
    SVHN_model = SVHN_ConvNet().to(device)
    SVHN_model.apply(weights_init)
    optimizer = optim.SGD(SVHN_model.parameters(), lr=pLearningRate, momentum=pMomentum, weight_decay=pWeightDecay)
    
    print("SVHN Training Started.....")
    memT,cpuT,gpuT = YN.StartLogger("Pytorch","SVHN") 
    start = time.time()
    for epoch in range(1,pEpochs + 1):  
        SVHN_train( SVHN_model, device, SVHN_train_loader, optimizer, epoch)
        SVHN_test(SVHN_model, device, SVHN_test_loader)
    end = time.time()
    print(str(datetime.timedelta(seconds=end-start)))
    YN.EndLogger(memT,cpuT,gpuT)
    print("SVHN Training Finished.....")
    
class SVHN_ConvNet(nn.Module):
    def __init__(self):
        super(SVHN_ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 48,  kernel_size=(5,5),padding=2,stride=1)
        self.conv2 = nn.Conv2d( 48, 64, kernel_size=(5, 5),padding=2,stride=1)
        self.conv3 = nn.Conv2d( 64, 128, kernel_size=(5, 5),padding=2,stride=1)
        self.conv4 = nn.Conv2d( 128, 160, kernel_size=(5, 5),padding=2,stride=1)
        self.conv5 = nn.Conv2d( 160, 192, kernel_size=(5, 5),padding=2,stride=1)
        self.conv6 = nn.Conv2d( 192, 192, kernel_size=(5, 5),padding=2,stride=1)
        self.conv7 = nn.Conv2d( 192, 192, kernel_size=(5, 5),padding=2,stride=1)
        self.conv8 = nn.Conv2d( 192, 192, kernel_size=(5, 5),padding=2,stride=1)
        
        self.bN0 = nn.BatchNorm2d(num_features=48)
        self.bN = nn.BatchNorm2d(num_features=64)
        self.bN1 = nn.BatchNorm2d(num_features=128)
        self.bN2 = nn.BatchNorm2d(num_features=160)
        self.bN3 = nn.BatchNorm2d(num_features=192)
        self.bN4 = nn.BatchNorm2d(num_features=192)
        self.bN5 = nn.BatchNorm2d(num_features=192)
        self.bN6 = nn.BatchNorm2d(num_features=192)
        
        self.fc1 = nn.Linear(192, 3072)
        self.fc2 = nn.Linear(3072, 10)

    def forward(self, x):
        
        x = self.conv1(x)
        x = self.bN0(x)
        x = F.relu(x)
        x = F.max_pool2d(x , kernel_size= (2, 2), stride=2, padding=1)
        x = F.dropout(x,0.2)
        
        x = self.conv2(x)
        x = self.bN(x)
        x = F.relu(x)
        x = F.max_pool2d(x , kernel_size= (2, 2), stride=2, padding=1)
        x = F.dropout(x,0.2)
        
        x = self.conv3(x)
        x = self.bN1(x)
        x = F.relu(x)
        x = F.max_pool2d(x , kernel_size= (2, 2), stride=2, padding=1)
        x = F.dropout(x,0.2)
        
        x = self.conv4(x)
        x = self.bN2(x)
        x = F.relu(x)
        x = F.max_pool2d(x , kernel_size= (2, 2), stride=2, padding=1)
        x = F.dropout(x,0.2)     
        
        x = self.conv5(x)
        x = self.bN3(x)
        x = F.relu(x)
        x = F.max_pool2d(x , kernel_size= (2, 2), stride=2, padding=1)
        x = F.dropout(x,0.2)
        
        x = self.conv6(x)
        x = self.bN4(x)
        x = F.relu(x)
        x = F.max_pool2d(x , kernel_size= (2, 2), stride=2, padding=1)
        x = F.dropout(x,0.2)
        
        x = self.conv7(x)
        x = self.bN5(x)
        x = F.relu(x)
        x = F.max_pool2d(x , kernel_size= (2, 2), stride=2, padding=1)
        x = F.dropout(x,0.2)
        
        x = self.conv8(x)
        x = self.bN6(x)
        x = F.relu(x)
        x = F.max_pool2d(x , kernel_size= (2, 2), stride=2)
        x = F.dropout(x,0.2)
        
        ###########        
        #Flatten  
        x = x.view(x.size(0), -1)
        x= F.relu(self.fc1(x))    
        x = self.fc2(x)
        y = F.log_softmax(x)
        return y
        
def runModel(dataset,batchSize=128,numClasses=10,epochs=12,learningRate=0.01,momentum=0.5,weightDecay=1e-6):
    if dataset is "mnist":
        RunMNIST(dataset,batchSize,numClasses,epochs,learningRate,momentum,weightDecay)
    elif dataset is "cifar10":
        RunCIFAR10(dataset,batchSize,numClasses,epochs,learningRate,momentum,weightDecay)
    elif dataset is "cifar100":
        RunCIFAR100(dataset,batchSize,numClasses,epochs,learningRate,momentum,weightDecay)   
    elif dataset is "SVHN":
        fname = './%s_32x32.mat'
        RunSVHN(dataset,batchSize,numClasses,epochs,learningRate,momentum,weightDecay)    
    else:
        print("Choose cifar10 or mnist or cifar100 or SVHN")


In [ ]:
def main():
    runModel("mnist",epochs=15)
    #runModel("cifar10",epochs=200)
    #runModel("cifar100",epochs=200)
    #runModel("SVHN",epochs=100)

In [ ]:
main()